In [1]:
import os
os.chdir("../../crystal/")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio

from tqdm import tqdm
from math import lcm
from typing import List, Dict
from ripser import Rips
from utils import wasserstein_distance_matrix, plot_distance_matrix

from Symmetry import Symmetry
from UnitCell import UnitCell
from RandomCrystal import RandomCrystal
from Fractional import FractionalCoordinate, FractionalCoordinateList
from Positional import PositionalCoordinate, PositionalCoordinateList

from sklearn.manifold import TSNE, Isomap, MDS, SpectralEmbedding
from sklearn.multiclass import OneVsOneClassifier
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

plt.rcParams['text.usetex'] = False

In [2]:
df = pd.read_csv("output/space_groups.csv", sep=" ")
df

,Space Group,Crystal System,Asymmetric Unit,Symmetries,Group Order,Unit Cell
0,2,Triclinic,30,"x,y,z;-x,-y,-z",2,"13.027,11.1822,9.2189,92.35,107.3,104.5"
1,9,Monoclinic,71,"x,y,z;x,-y,z+1/2;x+1/2,y+1/2,z;x+1/2,-y+1/2,z+1/2",4,"20.408,13.304,20.166,90.0,102.33,90.0"
2,10,Monoclinic,50,"x,y,z;-x,y,-z;-x,-y,-z;x,-y,z",4,"12.7411,12.6989,20.9991,90.0,96.29,90.0"
3,11,Monoclinic,25,"x,y,z;-x,y+1/2,-z;-x,-y,-z;x,-y+1/2,z",4,"11.454,21.695,7.227,90.0,93.15,90.0"
4,12,Monoclinic,24,"x,y,z;-x,y,-z;-x,-y,-z;x,-y,z;x+1/2,y+1/2,z;-x...",8,"22.684,13.373,12.553,90.0,69.48,90.0"
...,...,...,...,...,...,...
68,223,Cubic,7,"x,y,z;-x,-y,z;x,-y,-z;-x,y,-z;z,x,y;y,z,x;-z,-...",48,"13.705,13.705,13.705,90.0,90.0,90.0"
69,225,Cubic,4,"x,y,z;-x,-y,z;x,-y,-z;-x,y,-z;z,x,y;y,z,x;-z,-...",192,"13.624,13.624,13.624,90.0,90.0,90.0"
70,227,Cubic,5,"x,y,z;-x+1/4,-y+1/4,z;x,-y+1/4,-z+1/4;-x+1/4,y...",192,"24.345,24.345,24.345,90.0,90.0,90.0"
71,229,Cubic,5,"x,y,z;-x,-y,z;x,-y,-z;-x,y,-z;z,x,y;y,z,x;-z,-...",96,"18.578,18.578,18.578,90.0,90.0,90.0"


# Triclinic vs Monoclinic

In [28]:
crystal_system_1: str = 'Triclinic'
crystal_system_2: str = 'Monoclinic'

space_groups_1: np.ndarray = df.loc[df['Crystal System'] == crystal_system_1]['Space Group'].unique()
space_groups_2: np.ndarray = df.loc[df['Crystal System'] == crystal_system_2]['Space Group'].unique()

random_space_group_index: np.ndarray = np.random.randint(low = 0, high = len(space_groups_1), size = 1, dtype = int)[0]
space_group_1: int = space_groups_1[random_space_group_index]

random_space_group_index: np.ndarray = np.random.randint(low = 0, high = len(space_groups_2), size = 1, dtype = int)[0]
space_group_2: int = space_groups_2[random_space_group_index]

row_1 = df.loc[df['Space Group'] == space_group_1]
row_2 = df.loc[df['Space Group'] == space_group_2]

group_order_1: int = row_1['Group Order'].values[0]
group_order_2: int = row_2['Group Order'].values[0]

least_common_multiple: int = lcm(group_order_1, group_order_2)
print(f'Least common multiple: {least_common_multiple}')

crystal_dict = {}
n = 48

Least common multiple: 8


In [30]:
symmetries: list = row_1['Symmetries'].values[0].split(sep=";")
symmetries: List[Symmetry] = [Symmetry(sym) for sym in symmetries]

unit_cell: List[str] = row_1['Unit Cell'].values[0].split(sep=",")
unit_cell: UnitCell = UnitCell(*[float(x) for x in unit_cell])
normalised_unit_cell: UnitCell = unit_cell.normalise()

normalising_constant: float = unit_cell.normalising_constant
k = least_common_multiple // group_order_1
if least_common_multiple < n:
    k *= (n // least_common_multiple)

rips = Rips(maxdim=2, verbose=False)

for _ in tqdm(range(50)):
    random_crystal: RandomCrystal = RandomCrystal(symmetries, k)
    positional_coordinates: PositionalCoordinateList = random_crystal.fractional_coordinates.orthogonalise(unit_cell)
    normalised_positional_coordinates: PositionalCoordinateList = positional_coordinates.normalise(normalising_constant)

    distance_matrix: np.ndarray = normalised_positional_coordinates.calculate_distance_matrix(normalised_unit_cell, boundary_conditions=True)
    persistence = rips.fit_transform(X = distance_matrix, distance_matrix=True)

    for dim, intervals in enumerate(persistence):
        persistence[dim] = np.array(list(filter(lambda i: i[1] < float('inf'), intervals)))

    crystal_dict[f'{space_group_1}_{_+1}'] = {
        'system': crystal_system_1,
        'persistence': persistence
    }

100%|██████████| 50/50 [00:00<00:00, 84.45it/s]


In [31]:
symmetries: list = row_2['Symmetries'].values[0].split(sep=";")
symmetries: List[Symmetry] = [Symmetry(sym) for sym in symmetries]

unit_cell: List[str] = row_2['Unit Cell'].values[0].split(sep=",")
unit_cell: UnitCell = UnitCell(*[float(x) for x in unit_cell])
normalised_unit_cell: UnitCell = unit_cell.normalise()

normalising_constant: float = unit_cell.normalising_constant
k = least_common_multiple // group_order_2
if least_common_multiple < n:
    k *= (n // least_common_multiple)

rips = Rips(maxdim=2, verbose=False)

for _ in tqdm(range(50)):
    random_crystal: RandomCrystal = RandomCrystal(symmetries, k)
    positional_coordinates: PositionalCoordinateList = random_crystal.fractional_coordinates.orthogonalise(unit_cell)
    normalised_positional_coordinates: PositionalCoordinateList = positional_coordinates.normalise(normalising_constant)

    distance_matrix: np.ndarray = normalised_positional_coordinates.calculate_distance_matrix(normalised_unit_cell, boundary_conditions=True)
    persistence = rips.fit_transform(X = distance_matrix, distance_matrix=True)

    for dim, intervals in enumerate(persistence):
        persistence[dim] = np.array(list(filter(lambda i: i[1] < float('inf'), intervals)))

    crystal_dict[f'{space_group_2}_{_+1}'] = {
        'system': crystal_system_2,
        'persistence': persistence
    }

100%|██████████| 50/50 [00:00<00:00, 70.12it/s]


In [32]:
distance_matrix_0 = wasserstein_distance_matrix(crystal_dict, 0)
distance_matrix_1 = wasserstein_distance_matrix(crystal_dict, 1)
distance_matrix_2 = wasserstein_distance_matrix(crystal_dict, 2)

distance_matrix: np.ndarray = np.maximum.reduce([distance_matrix_0, distance_matrix_1, distance_matrix_2])

100%|██████████| 100/100 [00:01<00:00, 85.57it/s]


In [34]:
system = [v['system'] for v in crystal_dict.values()]
crystals = list(crystal_dict.keys())

n = 5
mds = MDS(n_components=n, dissimilarity='precomputed', metric=True)
embedding = mds.fit_transform(distance_matrix)

embedding_df = pd.DataFrame(embedding, columns=[str(_+1) for _ in range(n)])
embedding_df['Crystal System'] = system
embedding_df['Name'] = crystals

X = embedding
y = embedding_df['Crystal System'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

embedding_df.head()

,1,2,3,4,5,Crystal System,Name
0,0.165199,0.328322,-0.021521,0.265744,0.413502,Triclinic,2_1
1,-0.745970,-0.015208,0.517209,0.344020,-0.108280,Triclinic,2_2
2,0.207066,0.275433,-0.138255,0.283340,0.203409,Triclinic,2_3
3,-0.468655,-0.310115,0.099062,-0.119628,-0.071538,Triclinic,2_4
4,-0.698286,-0.155939,0.909205,-0.048793,0.358443,Triclinic,2_5


In [36]:
clf = OneVsOneClassifier(SVC())
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

class_report = classification_report(y_test, y_pred, target_names = embedding_df['Crystal System'].unique())
print('Classification Report:')
print(class_report)

Accuracy: 0.93
Classification Report:
              precision    recall  f1-score   support

   Triclinic       1.00      0.85      0.92        13
  Monoclinic       0.89      1.00      0.94        17

    accuracy                           0.93        30
   macro avg       0.95      0.92      0.93        30
weighted avg       0.94      0.93      0.93        30



# Triclinic vs Orthorhombic

In [38]:
crystal_system_1: str = 'Triclinic'
crystal_system_2: str = 'Orthorhombic'

space_groups_1: np.ndarray = df.loc[df['Crystal System'] == crystal_system_1]['Space Group'].unique()
space_groups_2: np.ndarray = df.loc[df['Crystal System'] == crystal_system_2]['Space Group'].unique()

random_space_group_index: np.ndarray = np.random.randint(low = 0, high = len(space_groups_1), size = 1, dtype = int)[0]
space_group_1: int = space_groups_1[random_space_group_index]

random_space_group_index: np.ndarray = np.random.randint(low = 0, high = len(space_groups_2), size = 1, dtype = int)[0]
space_group_2: int = space_groups_2[random_space_group_index]

row_1 = df.loc[df['Space Group'] == space_group_1]
row_2 = df.loc[df['Space Group'] == space_group_2]

group_order_1: int = row_1['Group Order'].values[0]
group_order_2: int = row_2['Group Order'].values[0]

least_common_multiple: int = lcm(group_order_1, group_order_2)
print(f'Least common multiple: {least_common_multiple}')

crystal_dict = {}
n = 48

Least common multiple: 16


In [39]:
symmetries: list = row_1['Symmetries'].values[0].split(sep=";")
symmetries: List[Symmetry] = [Symmetry(sym) for sym in symmetries]

unit_cell: List[str] = row_1['Unit Cell'].values[0].split(sep=",")
unit_cell: UnitCell = UnitCell(*[float(x) for x in unit_cell])
normalised_unit_cell: UnitCell = unit_cell.normalise()

normalising_constant: float = unit_cell.normalising_constant
k = least_common_multiple // group_order_1
if least_common_multiple < n:
    k *= (n // least_common_multiple)

rips = Rips(maxdim=2, verbose=False)

for _ in tqdm(range(50)):
    random_crystal: RandomCrystal = RandomCrystal(symmetries, k)
    positional_coordinates: PositionalCoordinateList = random_crystal.fractional_coordinates.orthogonalise(unit_cell)
    normalised_positional_coordinates: PositionalCoordinateList = positional_coordinates.normalise(normalising_constant)

    distance_matrix: np.ndarray = normalised_positional_coordinates.calculate_distance_matrix(normalised_unit_cell, boundary_conditions=True)
    persistence = rips.fit_transform(X = distance_matrix, distance_matrix=True)

    for dim, intervals in enumerate(persistence):
        persistence[dim] = np.array(list(filter(lambda i: i[1] < float('inf'), intervals)))

    crystal_dict[f'{space_group_1}_{_+1}'] = {
        'system': crystal_system_1,
        'persistence': persistence
    }

100%|██████████| 50/50 [00:00<00:00, 65.79it/s]


In [40]:
symmetries: list = row_2['Symmetries'].values[0].split(sep=";")
symmetries: List[Symmetry] = [Symmetry(sym) for sym in symmetries]

unit_cell: List[str] = row_2['Unit Cell'].values[0].split(sep=",")
unit_cell: UnitCell = UnitCell(*[float(x) for x in unit_cell])
normalised_unit_cell: UnitCell = unit_cell.normalise()

normalising_constant: float = unit_cell.normalising_constant
k = least_common_multiple // group_order_2
if least_common_multiple < n:
    k *= (n // least_common_multiple)

rips = Rips(maxdim=2, verbose=False)

for _ in tqdm(range(50)):
    random_crystal: RandomCrystal = RandomCrystal(symmetries, k)
    positional_coordinates: PositionalCoordinateList = random_crystal.fractional_coordinates.orthogonalise(unit_cell)
    normalised_positional_coordinates: PositionalCoordinateList = positional_coordinates.normalise(normalising_constant)

    distance_matrix: np.ndarray = normalised_positional_coordinates.calculate_distance_matrix(normalised_unit_cell, boundary_conditions=True)
    persistence = rips.fit_transform(X = distance_matrix, distance_matrix=True)

    for dim, intervals in enumerate(persistence):
        persistence[dim] = np.array(list(filter(lambda i: i[1] < float('inf'), intervals)))

    crystal_dict[f'{space_group_2}_{_+1}'] = {
        'system': crystal_system_2,
        'persistence': persistence
    }

100%|██████████| 50/50 [00:00<00:00, 52.88it/s]


In [41]:
distance_matrix_0 = wasserstein_distance_matrix(crystal_dict, 0)
distance_matrix_1 = wasserstein_distance_matrix(crystal_dict, 1)
distance_matrix_2 = wasserstein_distance_matrix(crystal_dict, 2)

distance_matrix: np.ndarray = np.maximum.reduce([distance_matrix_0, distance_matrix_1, distance_matrix_2])

100%|██████████| 100/100 [00:01<00:00, 83.31it/s]


In [42]:
system = [v['system'] for v in crystal_dict.values()]
crystals = list(crystal_dict.keys())

n = 5
mds = MDS(n_components=n, dissimilarity='precomputed', metric=True)
embedding = mds.fit_transform(distance_matrix)

embedding_df = pd.DataFrame(embedding, columns=[str(_+1) for _ in range(n)])
embedding_df['Crystal System'] = system
embedding_df['Name'] = crystals

X = embedding
y = embedding_df['Crystal System'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

embedding_df.head()

,1,2,3,4,5,Crystal System,Name
0,0.585855,0.238950,0.428096,0.485877,-0.055643,Triclinic,2_1
1,0.307327,-0.232716,1.070425,0.095646,-0.211679,Triclinic,2_2
2,0.424325,-0.206865,0.215732,0.101424,-0.118018,Triclinic,2_3
3,0.053621,-0.281255,0.878031,-0.046780,0.314145,Triclinic,2_4
4,0.214511,0.518470,0.863395,0.278534,-0.109135,Triclinic,2_5


In [43]:
clf = OneVsOneClassifier(SVC())
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

class_report = classification_report(y_test, y_pred, target_names = embedding_df['Crystal System'].unique())
print('Classification Report:')
print(class_report)

Accuracy: 0.97
Classification Report:
              precision    recall  f1-score   support

   Triclinic       1.00      0.92      0.96        13
Orthorhombic       0.94      1.00      0.97        17

    accuracy                           0.97        30
   macro avg       0.97      0.96      0.97        30
weighted avg       0.97      0.97      0.97        30



# Triclinic vs Tetragonal

In [45]:
crystal_system_1: str = 'Triclinic'
crystal_system_2: str = 'Tetragonal'

space_groups_1: np.ndarray = df.loc[df['Crystal System'] == crystal_system_1]['Space Group'].unique()
space_groups_2: np.ndarray = df.loc[df['Crystal System'] == crystal_system_2]['Space Group'].unique()

random_space_group_index: np.ndarray = np.random.randint(low = 0, high = len(space_groups_1), size = 1, dtype = int)[0]
space_group_1: int = space_groups_1[random_space_group_index]

random_space_group_index: np.ndarray = np.random.randint(low = 0, high = len(space_groups_2), size = 1, dtype = int)[0]
space_group_2: int = space_groups_2[random_space_group_index]

row_1 = df.loc[df['Space Group'] == space_group_1]
row_2 = df.loc[df['Space Group'] == space_group_2]

group_order_1: int = row_1['Group Order'].values[0]
group_order_2: int = row_2['Group Order'].values[0]

least_common_multiple: int = lcm(group_order_1, group_order_2)
print(f'Least common multiple: {least_common_multiple}')

crystal_dict = {}
n = 48

Least common multiple: 16


In [46]:
symmetries: list = row_1['Symmetries'].values[0].split(sep=";")
symmetries: List[Symmetry] = [Symmetry(sym) for sym in symmetries]

unit_cell: List[str] = row_1['Unit Cell'].values[0].split(sep=",")
unit_cell: UnitCell = UnitCell(*[float(x) for x in unit_cell])
normalised_unit_cell: UnitCell = unit_cell.normalise()

normalising_constant: float = unit_cell.normalising_constant
k = least_common_multiple // group_order_1
if least_common_multiple < n:
    k *= (n // least_common_multiple)

rips = Rips(maxdim=2, verbose=False)

for _ in tqdm(range(50)):
    random_crystal: RandomCrystal = RandomCrystal(symmetries, k)
    positional_coordinates: PositionalCoordinateList = random_crystal.fractional_coordinates.orthogonalise(unit_cell)
    normalised_positional_coordinates: PositionalCoordinateList = positional_coordinates.normalise(normalising_constant)

    distance_matrix: np.ndarray = normalised_positional_coordinates.calculate_distance_matrix(normalised_unit_cell, boundary_conditions=True)
    persistence = rips.fit_transform(X = distance_matrix, distance_matrix=True)

    for dim, intervals in enumerate(persistence):
        persistence[dim] = np.array(list(filter(lambda i: i[1] < float('inf'), intervals)))

    crystal_dict[f'{space_group_1}_{_+1}'] = {
        'system': crystal_system_1,
        'persistence': persistence
    }

100%|██████████| 50/50 [00:00<00:00, 83.93it/s]


In [47]:
symmetries: list = row_2['Symmetries'].values[0].split(sep=";")
symmetries: List[Symmetry] = [Symmetry(sym) for sym in symmetries]

unit_cell: List[str] = row_2['Unit Cell'].values[0].split(sep=",")
unit_cell: UnitCell = UnitCell(*[float(x) for x in unit_cell])
normalised_unit_cell: UnitCell = unit_cell.normalise()

normalising_constant: float = unit_cell.normalising_constant
k = least_common_multiple // group_order_2
if least_common_multiple < n:
    k *= (n // least_common_multiple)

rips = Rips(maxdim=2, verbose=False)

for _ in tqdm(range(50)):
    random_crystal: RandomCrystal = RandomCrystal(symmetries, k)
    positional_coordinates: PositionalCoordinateList = random_crystal.fractional_coordinates.orthogonalise(unit_cell)
    normalised_positional_coordinates: PositionalCoordinateList = positional_coordinates.normalise(normalising_constant)

    distance_matrix: np.ndarray = normalised_positional_coordinates.calculate_distance_matrix(normalised_unit_cell, boundary_conditions=True)
    persistence = rips.fit_transform(X = distance_matrix, distance_matrix=True)

    for dim, intervals in enumerate(persistence):
        persistence[dim] = np.array(list(filter(lambda i: i[1] < float('inf'), intervals)))

    crystal_dict[f'{space_group_2}_{_+1}'] = {
        'system': crystal_system_2,
        'persistence': persistence
    }

100%|██████████| 50/50 [00:01<00:00, 46.01it/s]


In [48]:
distance_matrix_0 = wasserstein_distance_matrix(crystal_dict, 0)
distance_matrix_1 = wasserstein_distance_matrix(crystal_dict, 1)
distance_matrix_2 = wasserstein_distance_matrix(crystal_dict, 2)

distance_matrix: np.ndarray = np.maximum.reduce([distance_matrix_0, distance_matrix_1, distance_matrix_2])

100%|██████████| 100/100 [00:01<00:00, 83.07it/s]


In [49]:
system = [v['system'] for v in crystal_dict.values()]
crystals = list(crystal_dict.keys())

n = 5
mds = MDS(n_components=n, dissimilarity='precomputed', metric=True)
embedding = mds.fit_transform(distance_matrix)

embedding_df = pd.DataFrame(embedding, columns=[str(_+1) for _ in range(n)])
embedding_df['Crystal System'] = system
embedding_df['Name'] = crystals

X = embedding
y = embedding_df['Crystal System'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

embedding_df.head()

,1,2,3,4,5,Crystal System,Name
0,-0.038503,-0.575677,1.078584,-0.151951,-0.297590,Triclinic,2_1
1,0.136689,-0.014051,0.861820,-0.131622,-0.105085,Triclinic,2_2
2,-0.474189,-0.497811,0.962369,0.318611,-0.602825,Triclinic,2_3
3,-0.227290,-0.290583,0.953831,-0.284974,-0.302349,Triclinic,2_4
4,0.201455,0.093092,0.895028,0.209212,-0.214282,Triclinic,2_5


In [50]:
clf = OneVsOneClassifier(SVC())
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

class_report = classification_report(y_test, y_pred, target_names = embedding_df['Crystal System'].unique())
print('Classification Report:')
print(class_report)

Accuracy: 0.90
Classification Report:
              precision    recall  f1-score   support

   Triclinic       1.00      0.77      0.87        13
  Tetragonal       0.85      1.00      0.92        17

    accuracy                           0.90        30
   macro avg       0.93      0.88      0.89        30
weighted avg       0.91      0.90      0.90        30



# Triclinic vs Trigonal

In [57]:
crystal_system_1: str = 'Triclinic'
crystal_system_2: str = 'Trigonal'

space_groups_1: np.ndarray = df.loc[df['Crystal System'] == crystal_system_1]['Space Group'].unique()
space_groups_2: np.ndarray = df.loc[df['Crystal System'] == crystal_system_2]['Space Group'].unique()

random_space_group_index: np.ndarray = np.random.randint(low = 0, high = len(space_groups_1), size = 1, dtype = int)[0]
space_group_1: int = space_groups_1[random_space_group_index]

random_space_group_index: np.ndarray = np.random.randint(low = 0, high = len(space_groups_2), size = 1, dtype = int)[0]
space_group_2: int = space_groups_2[random_space_group_index]

row_1 = df.loc[df['Space Group'] == space_group_1]
row_2 = df.loc[df['Space Group'] == space_group_2]

group_order_1: int = row_1['Group Order'].values[0]
group_order_2: int = row_2['Group Order'].values[0]

least_common_multiple: int = lcm(group_order_1, group_order_2)
print(f'Least common multiple: {least_common_multiple}')

crystal_dict = {}
n = 48

Least common multiple: 36


In [58]:
symmetries: list = row_1['Symmetries'].values[0].split(sep=";")
symmetries: List[Symmetry] = [Symmetry(sym) for sym in symmetries]

unit_cell: List[str] = row_1['Unit Cell'].values[0].split(sep=",")
unit_cell: UnitCell = UnitCell(*[float(x) for x in unit_cell])
normalised_unit_cell: UnitCell = unit_cell.normalise()

normalising_constant: float = unit_cell.normalising_constant
k = least_common_multiple // group_order_1
if least_common_multiple < n:
    k *= (n // least_common_multiple)

rips = Rips(maxdim=2, verbose=False)

for _ in tqdm(range(50)):
    random_crystal: RandomCrystal = RandomCrystal(symmetries, k)
    positional_coordinates: PositionalCoordinateList = random_crystal.fractional_coordinates.orthogonalise(unit_cell)
    normalised_positional_coordinates: PositionalCoordinateList = positional_coordinates.normalise(normalising_constant)

    distance_matrix: np.ndarray = normalised_positional_coordinates.calculate_distance_matrix(normalised_unit_cell, boundary_conditions=True)
    persistence = rips.fit_transform(X = distance_matrix, distance_matrix=True)

    for dim, intervals in enumerate(persistence):
        persistence[dim] = np.array(list(filter(lambda i: i[1] < float('inf'), intervals)))

    crystal_dict[f'{space_group_1}_{_+1}'] = {
        'system': crystal_system_1,
        'persistence': persistence
    }

100%|██████████| 50/50 [00:00<00:00, 159.59it/s]


In [59]:
symmetries: list = row_2['Symmetries'].values[0].split(sep=";")
symmetries: List[Symmetry] = [Symmetry(sym) for sym in symmetries]

unit_cell: List[str] = row_2['Unit Cell'].values[0].split(sep=",")
unit_cell: UnitCell = UnitCell(*[float(x) for x in unit_cell])
normalised_unit_cell: UnitCell = unit_cell.normalise()

normalising_constant: float = unit_cell.normalising_constant
k = least_common_multiple // group_order_2
if least_common_multiple < n:
    k *= (n // least_common_multiple)

rips = Rips(maxdim=2, verbose=False)

for _ in tqdm(range(50)):
    random_crystal: RandomCrystal = RandomCrystal(symmetries, k)
    positional_coordinates: PositionalCoordinateList = random_crystal.fractional_coordinates.orthogonalise(unit_cell)
    normalised_positional_coordinates: PositionalCoordinateList = positional_coordinates.normalise(normalising_constant)

    distance_matrix: np.ndarray = normalised_positional_coordinates.calculate_distance_matrix(normalised_unit_cell, boundary_conditions=True)
    persistence = rips.fit_transform(X = distance_matrix, distance_matrix=True)

    for dim, intervals in enumerate(persistence):
        persistence[dim] = np.array(list(filter(lambda i: i[1] < float('inf'), intervals)))

    crystal_dict[f'{space_group_2}_{_+1}'] = {
        'system': crystal_system_2,
        'persistence': persistence
    }

100%|██████████| 50/50 [00:00<00:00, 59.90it/s]


In [60]:
distance_matrix_0 = wasserstein_distance_matrix(crystal_dict, 0)
distance_matrix_1 = wasserstein_distance_matrix(crystal_dict, 1)
distance_matrix_2 = wasserstein_distance_matrix(crystal_dict, 2)

distance_matrix: np.ndarray = np.maximum.reduce([distance_matrix_0, distance_matrix_1, distance_matrix_2])

100%|██████████| 100/100 [00:01<00:00, 90.80it/s]


In [61]:
system = [v['system'] for v in crystal_dict.values()]
crystals = list(crystal_dict.keys())

n = 5
mds = MDS(n_components=n, dissimilarity='precomputed', metric=True)
embedding = mds.fit_transform(distance_matrix)

embedding_df = pd.DataFrame(embedding, columns=[str(_+1) for _ in range(n)])
embedding_df['Crystal System'] = system
embedding_df['Name'] = crystals

X = embedding
y = embedding_df['Crystal System'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

embedding_df.head()

,1,2,3,4,5,Crystal System,Name
0,-0.719584,-0.103654,-0.790742,-0.243752,-0.080248,Triclinic,2_1
1,-0.490912,0.008997,-0.587112,-0.517603,-0.166346,Triclinic,2_2
2,-0.933790,0.349426,-0.347401,-0.347485,0.395071,Triclinic,2_3
3,-0.839854,0.062510,-0.576279,-0.507213,-0.083270,Triclinic,2_4
4,-0.419921,0.264821,0.338313,-0.249635,0.113980,Triclinic,2_5


In [62]:
clf = OneVsOneClassifier(SVC())
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

class_report = classification_report(y_test, y_pred, target_names = embedding_df['Crystal System'].unique())
print('Classification Report:')
print(class_report)

Accuracy: 1.00
Classification Report:
              precision    recall  f1-score   support

   Triclinic       1.00      1.00      1.00        17
    Trigonal       1.00      1.00      1.00        13

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



# Triclinic vs Hexagonal

In [63]:
crystal_system_1: str = 'Triclinic'
crystal_system_2: str = 'Hexagonal'

space_groups_1: np.ndarray = df.loc[df['Crystal System'] == crystal_system_1]['Space Group'].unique()
space_groups_2: np.ndarray = df.loc[df['Crystal System'] == crystal_system_2]['Space Group'].unique()

random_space_group_index: np.ndarray = np.random.randint(low = 0, high = len(space_groups_1), size = 1, dtype = int)[0]
space_group_1: int = space_groups_1[random_space_group_index]

random_space_group_index: np.ndarray = np.random.randint(low = 0, high = len(space_groups_2), size = 1, dtype = int)[0]
space_group_2: int = space_groups_2[random_space_group_index]

row_1 = df.loc[df['Space Group'] == space_group_1]
row_2 = df.loc[df['Space Group'] == space_group_2]

group_order_1: int = row_1['Group Order'].values[0]
group_order_2: int = row_2['Group Order'].values[0]

least_common_multiple: int = lcm(group_order_1, group_order_2)
print(f'Least common multiple: {least_common_multiple}')

crystal_dict = {}
n = 48

Least common multiple: 24


In [64]:
symmetries: list = row_1['Symmetries'].values[0].split(sep=";")
symmetries: List[Symmetry] = [Symmetry(sym) for sym in symmetries]

unit_cell: List[str] = row_1['Unit Cell'].values[0].split(sep=",")
unit_cell: UnitCell = UnitCell(*[float(x) for x in unit_cell])
normalised_unit_cell: UnitCell = unit_cell.normalise()

normalising_constant: float = unit_cell.normalising_constant
k = least_common_multiple // group_order_1
if least_common_multiple < n:
    k *= (n // least_common_multiple)

rips = Rips(maxdim=2, verbose=False)

for _ in tqdm(range(50)):
    random_crystal: RandomCrystal = RandomCrystal(symmetries, k)
    positional_coordinates: PositionalCoordinateList = random_crystal.fractional_coordinates.orthogonalise(unit_cell)
    normalised_positional_coordinates: PositionalCoordinateList = positional_coordinates.normalise(normalising_constant)

    distance_matrix: np.ndarray = normalised_positional_coordinates.calculate_distance_matrix(normalised_unit_cell, boundary_conditions=True)
    persistence = rips.fit_transform(X = distance_matrix, distance_matrix=True)

    for dim, intervals in enumerate(persistence):
        persistence[dim] = np.array(list(filter(lambda i: i[1] < float('inf'), intervals)))

    crystal_dict[f'{space_group_1}_{_+1}'] = {
        'system': crystal_system_1,
        'persistence': persistence
    }

100%|██████████| 50/50 [00:00<00:00, 86.58it/s]


In [65]:
symmetries: list = row_2['Symmetries'].values[0].split(sep=";")
symmetries: List[Symmetry] = [Symmetry(sym) for sym in symmetries]

unit_cell: List[str] = row_2['Unit Cell'].values[0].split(sep=",")
unit_cell: UnitCell = UnitCell(*[float(x) for x in unit_cell])
normalised_unit_cell: UnitCell = unit_cell.normalise()

normalising_constant: float = unit_cell.normalising_constant
k = least_common_multiple // group_order_2
if least_common_multiple < n:
    k *= (n // least_common_multiple)

rips = Rips(maxdim=2, verbose=False)

for _ in tqdm(range(50)):
    random_crystal: RandomCrystal = RandomCrystal(symmetries, k)
    positional_coordinates: PositionalCoordinateList = random_crystal.fractional_coordinates.orthogonalise(unit_cell)
    normalised_positional_coordinates: PositionalCoordinateList = positional_coordinates.normalise(normalising_constant)

    distance_matrix: np.ndarray = normalised_positional_coordinates.calculate_distance_matrix(normalised_unit_cell, boundary_conditions=True)
    persistence = rips.fit_transform(X = distance_matrix, distance_matrix=True)

    for dim, intervals in enumerate(persistence):
        persistence[dim] = np.array(list(filter(lambda i: i[1] < float('inf'), intervals)))

    crystal_dict[f'{space_group_2}_{_+1}'] = {
        'system': crystal_system_2,
        'persistence': persistence
    }

100%|██████████| 50/50 [00:01<00:00, 47.78it/s]


In [66]:
distance_matrix_0 = wasserstein_distance_matrix(crystal_dict, 0)
distance_matrix_1 = wasserstein_distance_matrix(crystal_dict, 1)
distance_matrix_2 = wasserstein_distance_matrix(crystal_dict, 2)

distance_matrix: np.ndarray = np.maximum.reduce([distance_matrix_0, distance_matrix_1, distance_matrix_2])

100%|██████████| 100/100 [00:01<00:00, 85.70it/s]


In [67]:
system = [v['system'] for v in crystal_dict.values()]
crystals = list(crystal_dict.keys())

n = 5
mds = MDS(n_components=n, dissimilarity='precomputed', metric=True)
embedding = mds.fit_transform(distance_matrix)

embedding_df = pd.DataFrame(embedding, columns=[str(_+1) for _ in range(n)])
embedding_df['Crystal System'] = system
embedding_df['Name'] = crystals

X = embedding
y = embedding_df['Crystal System'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

embedding_df.head()

,1,2,3,4,5,Crystal System,Name
0,0.655634,0.295137,0.068082,-0.628344,1.018221,Triclinic,2_1
1,-0.041008,0.015500,0.494180,-1.270732,0.655032,Triclinic,2_2
2,0.602528,-0.072387,0.042657,-1.051711,0.749787,Triclinic,2_3
3,-0.340024,-0.035990,0.632445,-0.349320,-0.083834,Triclinic,2_4
4,0.159580,-0.014740,-0.003095,-0.569111,1.006888,Triclinic,2_5


In [68]:
clf = OneVsOneClassifier(SVC())
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

class_report = classification_report(y_test, y_pred, target_names = embedding_df['Crystal System'].unique())
print('Classification Report:')
print(class_report)

Accuracy: 1.00
Classification Report:
              precision    recall  f1-score   support

   Triclinic       1.00      1.00      1.00        13
   Hexagonal       1.00      1.00      1.00        17

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



# Triclinic vs Cubic

In [69]:
crystal_system_1: str = 'Triclinic'
crystal_system_2: str = 'Cubic'

space_groups_1: np.ndarray = df.loc[df['Crystal System'] == crystal_system_1]['Space Group'].unique()
space_groups_2: np.ndarray = df.loc[df['Crystal System'] == crystal_system_2]['Space Group'].unique()

random_space_group_index: np.ndarray = np.random.randint(low = 0, high = len(space_groups_1), size = 1, dtype = int)[0]
space_group_1: int = space_groups_1[random_space_group_index]

random_space_group_index: np.ndarray = np.random.randint(low = 0, high = len(space_groups_2), size = 1, dtype = int)[0]
space_group_2: int = space_groups_2[random_space_group_index]

row_1 = df.loc[df['Space Group'] == space_group_1]
row_2 = df.loc[df['Space Group'] == space_group_2]

group_order_1: int = row_1['Group Order'].values[0]
group_order_2: int = row_2['Group Order'].values[0]

least_common_multiple: int = lcm(group_order_1, group_order_2)
print(f'Least common multiple: {least_common_multiple}')

crystal_dict = {}
n = 48

Least common multiple: 24


In [70]:
symmetries: list = row_1['Symmetries'].values[0].split(sep=";")
symmetries: List[Symmetry] = [Symmetry(sym) for sym in symmetries]

unit_cell: List[str] = row_1['Unit Cell'].values[0].split(sep=",")
unit_cell: UnitCell = UnitCell(*[float(x) for x in unit_cell])
normalised_unit_cell: UnitCell = unit_cell.normalise()

normalising_constant: float = unit_cell.normalising_constant
k = least_common_multiple // group_order_1
if least_common_multiple < n:
    k *= (n // least_common_multiple)

rips = Rips(maxdim=2, verbose=False)

for _ in tqdm(range(50)):
    random_crystal: RandomCrystal = RandomCrystal(symmetries, k)
    positional_coordinates: PositionalCoordinateList = random_crystal.fractional_coordinates.orthogonalise(unit_cell)
    normalised_positional_coordinates: PositionalCoordinateList = positional_coordinates.normalise(normalising_constant)

    distance_matrix: np.ndarray = normalised_positional_coordinates.calculate_distance_matrix(normalised_unit_cell, boundary_conditions=True)
    persistence = rips.fit_transform(X = distance_matrix, distance_matrix=True)

    for dim, intervals in enumerate(persistence):
        persistence[dim] = np.array(list(filter(lambda i: i[1] < float('inf'), intervals)))

    crystal_dict[f'{space_group_1}_{_+1}'] = {
        'system': crystal_system_1,
        'persistence': persistence
    }

100%|██████████| 50/50 [00:00<00:00, 72.77it/s]


In [71]:
symmetries: list = row_2['Symmetries'].values[0].split(sep=";")
symmetries: List[Symmetry] = [Symmetry(sym) for sym in symmetries]

unit_cell: List[str] = row_2['Unit Cell'].values[0].split(sep=",")
unit_cell: UnitCell = UnitCell(*[float(x) for x in unit_cell])
normalised_unit_cell: UnitCell = unit_cell.normalise()

normalising_constant: float = unit_cell.normalising_constant
k = least_common_multiple // group_order_2
if least_common_multiple < n:
    k *= (n // least_common_multiple)

rips = Rips(maxdim=2, verbose=False)

for _ in tqdm(range(50)):
    random_crystal: RandomCrystal = RandomCrystal(symmetries, k)
    positional_coordinates: PositionalCoordinateList = random_crystal.fractional_coordinates.orthogonalise(unit_cell)
    normalised_positional_coordinates: PositionalCoordinateList = positional_coordinates.normalise(normalising_constant)

    distance_matrix: np.ndarray = normalised_positional_coordinates.calculate_distance_matrix(normalised_unit_cell, boundary_conditions=True)
    persistence = rips.fit_transform(X = distance_matrix, distance_matrix=True)

    for dim, intervals in enumerate(persistence):
        persistence[dim] = np.array(list(filter(lambda i: i[1] < float('inf'), intervals)))

    crystal_dict[f'{space_group_2}_{_+1}'] = {
        'system': crystal_system_2,
        'persistence': persistence
    }

100%|██████████| 50/50 [00:01<00:00, 36.61it/s]


In [72]:
distance_matrix_0 = wasserstein_distance_matrix(crystal_dict, 0)
distance_matrix_1 = wasserstein_distance_matrix(crystal_dict, 1)
distance_matrix_2 = wasserstein_distance_matrix(crystal_dict, 2)

distance_matrix: np.ndarray = np.maximum.reduce([distance_matrix_0, distance_matrix_1, distance_matrix_2])

100%|██████████| 100/100 [00:01<00:00, 74.04it/s]


In [73]:
system = [v['system'] for v in crystal_dict.values()]
crystals = list(crystal_dict.keys())

n = 5
mds = MDS(n_components=n, dissimilarity='precomputed', metric=True)
embedding = mds.fit_transform(distance_matrix)

embedding_df = pd.DataFrame(embedding, columns=[str(_+1) for _ in range(n)])
embedding_df['Crystal System'] = system
embedding_df['Name'] = crystals

X = embedding
y = embedding_df['Crystal System'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

embedding_df.head()

,1,2,3,4,5,Crystal System,Name
0,0.374403,0.009753,-0.110494,0.371849,-0.244655,Triclinic,2_1
1,-0.151028,-0.415694,0.005568,-0.066634,0.204762,Triclinic,2_2
2,-0.243043,-0.351292,0.156409,0.005634,-0.620960,Triclinic,2_3
3,0.433591,0.023117,-0.042956,0.090754,-0.149337,Triclinic,2_4
4,-0.818204,0.117764,0.183034,-0.161103,0.114827,Triclinic,2_5


In [74]:
clf = OneVsOneClassifier(SVC())
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

class_report = classification_report(y_test, y_pred, target_names = embedding_df['Crystal System'].unique())
print('Classification Report:')
print(class_report)

Accuracy: 1.00
Classification Report:
              precision    recall  f1-score   support

   Triclinic       1.00      1.00      1.00        13
       Cubic       1.00      1.00      1.00        17

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30

